## Animation

Environment

In [1]:
import sys
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")

from ronek import env
env.set(
  device="cpu",
  device_idx=0,
  nb_threads=8,
  floatx="float64"
)

Import libraries

In [2]:
import os
import numpy as np
import dill as pickle

from ronek.systems import TASystem
from ronek.postproc import animate_dist
from silx.io.dictdump import h5todict

In [3]:
import matplotlib.pyplot as plt
stylefile = "/home/zanardi/Workspace/Research/styles/matplotlib/paper_1column.mplstyle"
plt.style.use(stylefile)

Inputs

In [4]:
rom_dim = 32
tlim = [6e-8, 1e-5]
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/RVC_O3/database/"
path_to_fom = "/home/zanardi/Codes/ML/RONEK/run/RVC_O3/test/data/case_cold_T10000.p"
path_to_bases = "/home/zanardi/Codes/ML/RONEK/run/RVC_O3/test/max_mom_10/cobras_bases.hdf5"
path_to_saving = "./animations/"

In [5]:
os.makedirs(path_to_saving, exist_ok=True)

Initialize isothermal master equation model

In [6]:
system = TASystem(
  species={
    k: path_to_dtb + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  rates_coeff=path_to_dtb + "/kinetics.hdf5",
  use_einsum=False,
  use_factorial=True,
  use_arrhenius=True
)

Load FOM solution and bases

In [7]:
data = pickle.load(open(path_to_fom, "rb"))
T, t, n0, n_fom = [data[k] for k in ("T", "t", "n0", "n")]
si, ei = [np.argmin(np.abs(t-ti)) for ti in tlim]

In [8]:
bases = h5todict(path_to_bases)

ROM solution

In [9]:
system.update_fom_ops(T)
system.update_rom_ops(
  phi=bases["phi"][:,:rom_dim],
  psi=bases["psi"][:,:rom_dim]
)

In [10]:
sols = {
  "FOM": n_fom[1][:,si:ei],
  "ROM": system.solve_rom(t, n0)[1][:,si:ei]
}

Animate

In [ ]:
animate_dist(
  path=path_to_saving,
  t=t[si:ei],
  n_m=sols,
  molecule=system.mix.species["molecule"],
  markersize=1
)